In [1]:
import os
import cv2
import collections
import time
import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, train_test_split

import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
import configparser
import argparse
import wandb

# Catalyst is amazing.
from catalyst.data import Augmentor
from catalyst.dl import utils
from catalyst.data.reader import ImageReader, ScalarReader, ReaderCompose, LambdaReader
from catalyst.dl.runner import SupervisedRunner
# from catalyst.dl.runner import SupervisedWandbRunner as SupervisedRunner
from catalyst.contrib.models.segmentation import Unet
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback

# PyTorch made my work much much easier.
import segmentation_models_pytorch as smp
from dataloader import SegmentationDataset, SegmentationDatasetTest, SegmentationDataset_withid
from augmentations import get_training_augmentation, get_preprocessing
from augmentations import get_test_augmentation, get_validation_augmentation

from utils import *
from metric import dice
import pickle

In [3]:
def get_ids(train_ids_file='../train_ids.pkl', valid_ids_file='../valid_ids.pkl'):
    with open(train_ids_file, 'rb') as handle:
        train_ids = pickle.load(handle)

    with open(valid_ids_file, 'rb') as handle:
        valid_ids = pickle.load(handle)

    return train_ids, valid_ids

train_ids, valid_ids = get_ids()
# valid_ids = list(train_ids)+list(valid_ids)


# FIX LOADERS

def get_loaders(bs=32, num_workers=4, preprocessing_fn=None,
            img_db="../input/train_images_480/", mask_db="../input/train_masks_480/",
            npy=True):
        train_ids, valid_ids = get_ids()

        train_dataset = SegmentationDataset(ids=train_ids,
                    transforms=get_training_augmentation(),
                    preprocessing=get_preprocessing(preprocessing_fn),
                    img_db=img_db,
                    mask_db=mask_db, npy=npy)
        valid_dataset = SegmentationDataset(ids=valid_ids,
                    transforms=get_validation_augmentation(),
                    preprocessing=get_preprocessing(preprocessing_fn),
                    img_db=img_db,
                    mask_db=mask_db, npy=npy)

        train_loader = DataLoader(train_dataset, batch_size=bs,
            shuffle=True, num_workers=num_workers)
        valid_loader = DataLoader(valid_dataset, batch_size=bs,
            shuffle=False, num_workers=num_workers)

        loaders = {
            "train": train_loader,
            "valid": valid_loader
        }
        return valid_dataset, loaders
    
bs = 8    
num_workers = 0
encoder = 'efficientnet-b4'
arch = 'unet'
model, preprocessing_fn = get_model(encoder, type=arch)
valid_dataset, loaders = get_loaders(bs, num_workers, preprocessing_fn)
train_loader = loaders['train']
valid_loader = loaders['valid']

# model, preprocessing_fn = get_model(encoder)
# loaders = get_loaders(bs, num_workers, preprocessing_fn)

Training on unet architecture with efficientnet-b4 encoder


/opt/anaconda3/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2567: UserWarning:

Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().



In [4]:
print("Loading model")
model_path = f"../logs/unet_efficientnet-b4/checkpoints/best.pth"

runner = SupervisedRunner()
encoded_pixels = []
loaders = {"infer": valid_loader}
runner.infer(
    model=model,
    loaders=loaders,
    callbacks=[
        CheckpointCallback(
            resume=model_path),
        InferCallback()
    ],
)
loaders['train'] = train_loader
loaders['valid'] = valid_loader

Loading model
=> loading checkpoint ../logs/unet_efficientnet-b4/checkpoints/best.pth
loaded checkpoint ../logs/unet_efficientnet-b4/checkpoints/best.pth (epoch 17)
Top best models:



In [5]:
valid_masks = []
LIMIT = 800
probabilities = np.zeros((int(LIMIT*4), 320, 480)) #HARDCODED FOR NOW
for i, (batch, output) in enumerate(tqdm.tqdm(zip(valid_dataset, runner.callbacks[0].predictions["logits"]))):
        if i >= LIMIT:
            break
        image, mask = batch
        for m in mask:
            # if m.shape != (350, 525):
            #     m = cv2.resize(m, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
            valid_masks.append(m)

        for j, probability in enumerate(output):
            # if probability.shape != (350, 525):
            #     probability = cv2.resize(probability, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
            probabilities[i * 4 + j, :, :] = probability

800it [02:46,  4.34it/s]

# My new method

In [6]:
def post_process(probability, threshold, min_size, 
                 threshold_type='percentile', size=(350, 525)):
    """
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    # don't remember where I saw it
    if threshold_type == 'mean':
        threshold = np.mean(probability)
    elif threshold_type == 'percentile':    
        threshold = np.percentile(probability, threshold)    
        
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros(size, np.float32)
    num = 0
    
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

def does_not_have(img_name, class_id, df):
    if class_id in df[df.img_name==img_name].label.values:
        return False
    return True

In [7]:
CLASSES = ['Fish', 'Flower', 'Gravel', 'Sugar']
MAPPING = dict(zip(CLASSES, [x for x in range(4)]))
_, valid_ids = get_ids()
valid_ids = valid_ids[:LIMIT]

In [8]:
df = pd.read_csv('../input/train.csv')
df = df[~df.EncodedPixels.isna()]
df['img_name'] = df.Image_Label.apply(lambda x: x.split('_')[0])
df['label'] = df.Image_Label.apply(lambda x: MAPPING[x.split('_')[1]])
df.drop(['Image_Label', 'EncodedPixels'], axis=1, inplace=True)
df.head()

,img_name,label
0,0011165.jpg,0
1,0011165.jpg,1
4,002be4f.jpg,0
5,002be4f.jpg,1
7,002be4f.jpg,3


In [9]:
size = (320, 480)
class_params = {}
sigmoid = lambda x: 1 / (1 + np.exp(-x))
for class_id in range(4):
        print(class_id)
        attempts = []
        for t in tqdm.tqdm(range(0, 100, 5)):
#             t /= 100
            for ms in [5000, 10000, 15000, 20000, 25000, 27000]:
                masks = []
                for i, img_name in zip(range(class_id, len(probabilities), 4), valid_ids):
                    if does_not_have(img_name, class_id, df):
                        predict = np.zeros(size)
                    else:    
                        probability = probabilities[i]
                        predict, num_predict = post_process(sigmoid(probability), t,
                                                            ms, size=size)
                    masks.append(predict)
                d = []
                for i, j in zip(masks, valid_masks[class_id::4]):
                    if (i.sum() == 0) & (j.sum() == 0):
                        d.append(1)
                    else:
                        d.append(dice(i, j))
                attempts.append((t, ms, np.mean(d)))

        attempts_df = pd.DataFrame(attempts, columns=['threshold', 'size', 'dice'])
        attempts_df = attempts_df.sort_values('dice', ascending=False)
        print(attempts_df.head())
        best_threshold = attempts_df['threshold'].values[0]
        best_size = attempts_df['size'].values[0]

        class_params[class_id] = (best_threshold, best_size)
print(class_params)        


  0%|          | 0/20 [00:00<?, ?it/s]

0



100%|██████████| 20/20 [06:45<00:00, 20.25s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

    threshold   size      dice
28         20  25000  0.720396
33         25  20000  0.720247
29         20  27000  0.720214
27         20  20000  0.719970
32         25  15000  0.719828
1



100%|██████████| 20/20 [06:16<00:00, 18.82s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

    threshold   size      dice
23         15  27000  0.769522
22         15  25000  0.769481
16         10  25000  0.769471
17         10  27000  0.769471
13         10  10000  0.769459
2



100%|██████████| 20/20 [07:09<00:00, 21.46s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

    threshold   size      dice
23         15  27000  0.701414
29         20  27000  0.700510
22         15  25000  0.700453
21         15  20000  0.700386
27         20  20000  0.700313
3



100%|██████████| 20/20 [07:35<00:00, 22.78s/it]

    threshold   size      dice
27         20  20000  0.575085
29         20  27000  0.574940
23         15  27000  0.574555
31         25  10000  0.574517
26         20  15000  0.574498
{0: (20, 25000), 1: (15, 27000), 2: (15, 27000), 3: (20, 20000)}


# Make answer

In [1]:
import os
import cv2
import collections
import time
import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, train_test_split

import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
import configparser
import argparse
import wandb

# Catalyst is amazing.
from catalyst.data import Augmentor
from catalyst.dl import utils
from catalyst.data.reader import ImageReader, ScalarReader, ReaderCompose, LambdaReader
from catalyst.dl.runner import SupervisedRunner
# from catalyst.dl.runner import SupervisedWandbRunner as SupervisedRunner
from catalyst.contrib.models.segmentation import Unet
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback

# PyTorch made my work much much easier.
import segmentation_models_pytorch as smp
from dataloader import SegmentationDataset, SegmentationDatasetTest, SegmentationDataset_withid
from augmentations import get_training_augmentation, get_preprocessing
from augmentations import get_test_augmentation, get_validation_augmentation

from utils import *
from metric import dice
import pickle

class_params = {0: (20, 25000), 1: (15, 27000), 2: (15, 27000), 3: (20, 20000)}

def post_process(probability, threshold, min_size, 
                 threshold_type='percentile', size=(350, 525)):
    """
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    # don't remember where I saw it
    if threshold_type == 'mean':
        threshold = np.mean(probability)
    elif threshold_type == 'percentile':    
        threshold = np.percentile(probability, threshold)    
        
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros(size, np.float32)
    num = 0
    
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num


def get_ids(train_ids_file='../train_ids.pkl', valid_ids_file='../valid_ids.pkl'):
    with open(train_ids_file, 'rb') as handle:
        train_ids = pickle.load(handle)

    with open(valid_ids_file, 'rb') as handle:
        valid_ids = pickle.load(handle)

    return train_ids, valid_ids

def get_loaders(bs=32, num_workers=4, preprocessing_fn=None,
            img_db="../input/train_images_480/", mask_db="../input/train_masks_480/",
            npy=True):
        train_ids, valid_ids = get_ids()

        valid_ids = valid_ids[:100]

        valid_dataset = SegmentationDataset(ids=valid_ids,
                    transforms=get_validation_augmentation(),
                    preprocessing=get_preprocessing(preprocessing_fn),
                    img_db=img_db,
                    mask_db=mask_db, npy=npy)

        valid_loader = DataLoader(valid_dataset, batch_size=bs,
            shuffle=False, num_workers=num_workers)

        loaders = {
            "infer": valid_loader
        }
        return loaders




sigmoid = lambda x: 1 / (1 + np.exp(-x))

bs = 4
num_workers = 0
encoder = 'efficientnet-b4'
arch = 'unet'
model, preprocessing_fn = get_model(encoder, type=arch)
model_path = f"../logs/unet_efficientnet-b4/checkpoints/best.pth"


import gc
torch.cuda.empty_cache()
gc.collect()

sub = pd.read_csv(f'../input/sample_submission.csv')
sub['label'] = sub['Image_Label'].apply(lambda x: x.split('_')[1])
sub['im_id'] = sub['Image_Label'].apply(lambda x: x.split('_')[0])
test_ids = sub['Image_Label'].apply(lambda x: x.split('_')[0]).drop_duplicates().values

# Load model, weird way in catalyst
loaders = get_loaders()
# checkpoint = torch.load(model_path)
# model.load_state_dict(checkpoint['model_state_dict'])

runner = SupervisedRunner()
runner.infer(
    model=model,
    loaders=loaders,
    callbacks=[
        CheckpointCallback(
            resume=model_path),
        InferCallback()
    ],
)

test_dataset = SegmentationDatasetTest(test_ids,
                                        transforms=get_test_augmentation(),
                                        preprocessing=get_preprocessing(preprocessing_fn),
                                        img_db="../input/test_images_525/test_images_525")

test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False,
                            num_workers=num_workers)

loaders = {"test": test_loader}

# {0: (20, 25000), 1: (15, 27000), 2: (15, 27000), 3: (20, 20000)}

encoded_pixels = []
image_id = 0
size = (350, 525) #Required output size by kaggle
for i, test_batch in enumerate(tqdm.tqdm(loaders['test'])):
    runner_out = runner.predict_batch({"features": test_batch.cuda()})['logits']
    for i, batch in enumerate(runner_out):
        for probability in batch:
            probability = probability.cpu().detach().numpy()
            if probability.shape != (350, 525):
                probability = cv2.resize(probability, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
            predict, num_predict = post_process(sigmoid(probability),
                                                class_params[image_id % 4][0],
                                                class_params[image_id % 4][1],
                                                size=size)
            if num_predict == 0:
                encoded_pixels.append('')
            else:
                r = mask2rle(predict)
                encoded_pixels.append(r)
            image_id += 1

sub['EncodedPixels'] = encoded_pixels

# Use classifer
import pickle
with open('../list.pkl', 'rb') as handle:
    image_labels_empty = pickle.load(handle)

predictions_nonempty = set(sub.loc[~sub['EncodedPixels'].isnull(), 'Image_Label'].values)
print(f'{len(image_labels_empty.intersection(predictions_nonempty))} masks would be removed')

sub.loc[sub['Image_Label'].isin(image_labels_empty), 'EncodedPixels'] = np.nan
sub.to_csv("UnetEffnet-b4WithMyPP.csv", columns=['Image_Label', 'EncodedPixels'], index=False)


# git fetch --all && git reset --hard origin/master

Training on unet architecture with efficientnet-b4 encoder


/opt/anaconda3/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2567: UserWarning:

Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().



=> loading checkpoint ../logs/unet_efficientnet-b4/checkpoints/best.pth
loaded checkpoint ../logs/unet_efficientnet-b4/checkpoints/best.pth (epoch 17)


  0%|          | 0/925 [00:00<?, ?it/s]

Top best models:



100%|██████████| 925/925 [02:57<00:00,  5.20it/s]


3301 masks would be removed


In [2]:
sub.head()

,Image_Label,EncodedPixels,label,im_id
0,002f507.jpg_Fish,NaN,Fish,002f507.jpg
1,002f507.jpg_Flower,NaN,Flower,002f507.jpg
2,002f507.jpg_Gravel,3 344 352 346 701 349 1051 65099 66151 349 665...,Gravel,002f507.jpg
3,002f507.jpg_Sugar,NaN,Sugar,002f507.jpg
4,0035ae9.jpg_Fish,71 143 432 46 504 16 776 135 1125 138 1475 139...,Fish,0035ae9.jpg
